In [2]:
import json
import os
from sklearn.model_selection import train_test_split

In [12]:
#Unique chars: {'f', '✋', 'с', 'c', 'н', 'и', '_', '¼', '5', '6', '8', '&', 'ш', '¾', 'b', 'o', 'm', 'w', 'е', ':', 'e', 'т', '=', '’', 'ı', '❌', '!', '”', 'о', 't', 'i', 'n', 'x', '⠀', 'é', '.', 'j', 'a', '–', "'", 'h', 'p', '\u200b', '3', '−', '\xad', '\u200e', '(', '%', '@', '©', 'ʻ', '|', '☺', 'м', '—', 'ь', '"', 's', '‑', 'z', '′', 'r', '[', '️', 'q', '1', '-', 'k', 'u', '#', 'v', 'y', '•', ';', '>', 'ʼ', 'd', '\ufeff', '»', 'к', '`', 'ş', '9', 'ò', '?', ')', '+', '…', '2', ',', 'р', 'ä', '7', '“', 'l', '―', '→', '«', 'й', '®', 'г', ']', '\r', 'ü', '‘', '/', 'у', '0', 'а', '4', ' ', 'х', '¬', 'ñ', 'g'}

#function replace crillic chars with latin chars
def replace_cyrillic(text):
    cyrillic = ['а', 'б', 'в', 'г', 'д', 'е', 'ё', 'ж', 'з', 'и',
                'й', 'к', 'л', 'м', 'н', 'о', 'п', 'р', 'с', 'т',
                'у', 'ф', 'х', 'ц', 'ч', 'ш', 'щ', 'ъ', 'ы', 'ь',
                'э', 'ю', 'я', 'қ', 'ғ', 'ҳ']
    latin = ['a', 'b', 'v', 'g', 'd', 'e', 'yo', 'j', 'z', 'i',
                'y', 'k', 'l', 'm', 'n', 'o', 'p', 'r', 's', 't',
                'u', 'f', 'x', 'ts', 'ch', 'sh', 'sch', '', 'i', '',
                'e', 'yu', 'ya', 'q', 'g\'', 'h']
    for i in range(len(cyrillic)):
        text = text.replace(cyrillic[i], latin[i])
    return text


In [27]:
#replace numiric chars with word
def replace_numiric(text):
    numiric = ['1', '2', '3', '4', '5', '6', '7', '8', '9', '0']
    numiric_word=['bir', 'ikki', 'uch', 'to‘rt', 'besh', 'olti', 'yetti', 'sakkiz', 'to\'qqiz', 'nol']

    for i in range(len(numiric)):
        text = text.replace(numiric[i], numiric_word[i])
        

In [28]:
#function replace special chars with space
def replace_special(text):
    uknwon=  [ '✋', 'с', 'н', 'и', '_', '¼',  '&', 'ш', '¾', 'w', ':', 'т', '=', 'ı', '❌', '!', '”', 'о', '⠀', 'é', '.', '–', '\u200b', '−', '\xad', '\u200e', '(', '%', '@', '©', '|', '☺', 'м', '—', 'ь', '"',  '‑', '[', '-', '#', '•', ';', '>', '\ufeff', '»', 'к', 'ş', 'ò', '?', ')', '+', '…', ',', 'р', 'ä',  '“', '―', '→', '«', 'й', '®', 'г', ']', '\r', 'ü', '/', 'у',  'а', ' ', 'х', '¬', 'ñ',]
    for i in range(len(uknwon)):
        text = text.replace(uknwon[i], ' ')
    return text

In [29]:
#{'f', '✋', 'с', 'c', 'н', 'и', '_', '¼', '5', '6', '8', '&', 'ш', '¾', 'b', 'o', 'm', 'w', 'е', ':', 'e', 'т', '=', '’', 'ı', '❌', '!', '”', 'о', 't', 'i', 'n', 'x', '⠀', 'é', '.', 'j', 'a', '–', "'", 'h', 'p', '\u200b', '3', '−', '\xad', '\u200e', '(', '%', '@', '©', 'ʻ', '|', '☺', 'м', '—', 'ь', '"', 's', '‑', 'z', '′', 'r', '[', '️', 'q', '1', '-', 'k', 'u', '#', 'v', 'y', '•', ';', '>', 'ʼ', 'd', '\ufeff', '»', 'к', '`', 'ş', '9', 'ò', '?', ')', '+', '…', '2', ',', 'р', 'ä', '7', '“', 'l', '―', '→', '«', 'й', '®', 'г', ']', '\r', 'ü', '‘', '/', 'у', '0', 'а', '4', ' ', 'х', '¬', 'ñ', 'g'}

#replace special "'" like chars with "'"
def replace_special_apostrophe(text):
    uknwon=  [ '’', '‘', 'ʻ', 'ʼ', '`', '’', '′']
    for i in range(len(uknwon)):
        text = text.replace(uknwon[i], '\'')
    return text

In [30]:
def process_and_split_json(input_filepath, train_output_filepath, test_output_filepath, test_size=0.2):
    with open(input_filepath, 'r', encoding='utf-8') as input_file:
        data = input_file.read()
        data = json.loads(data)  # Remove the 'encoding' parameter
    
    print(f"Total data size: {len(data)}")
    # Filter out entries where is_valid is 0
    valid_data = [entry for entry in data if entry["is_valid"] == 1]
    print(f"Total valid data size: {len(valid_data)}")
    train_data, test_data = train_test_split(valid_data, test_size=test_size, random_state=42)
    unique_chars = set()
    unique_chars_after = set()
    def create_new_entry(entry):
        audio_filepath = f"/media/real/data/uzbekvoice/clips/{entry['client_id']}/{entry['original_sentence_id']}.wav"
        if os.path.exists(audio_filepath):
            unique_chars.update(entry["original_sentence"].lower())
            unique_chars_after.update(replace_special(replace_cyrillic(replace_special_apostrophe((entry["original_sentence"].lower()))) ))
            return {
                "audio_filepath": audio_filepath,
                "duration": entry["clip_duration"],
                "text": replace_special(replace_cyrillic(replace_special_apostrophe((entry["original_sentence"].lower()))) ),
            }
        else:
            print(f"Could not find audio file: {audio_filepath}")
            return None
    
    train_new_data = [create_new_entry(entry) for entry in train_data]
    test_new_data = [create_new_entry(entry) for entry in test_data]
    
    train_new_data = [entry for entry in train_new_data if entry is not None]
    test_new_data = [entry for entry in test_new_data if entry is not None]
    
    print(f"Unique chars: {unique_chars}")
    print(f"Unique chars after: {unique_chars_after}")
    print(f"Train size: {len(train_new_data)}")
    print(f"Test size: {len(test_new_data)}")
    with open(train_output_filepath, 'w', encoding='utf-8') as train_output_file:
        for entry in train_new_data:
            json.dump(entry, train_output_file, ensure_ascii=False)
            train_output_file.write('\n')
    
    with open(test_output_filepath, 'w', encoding='utf-8') as test_output_file:
        for entry in test_new_data:
            json.dump(entry, test_output_file, ensure_ascii=False)
            test_output_file.write('\n')


In [31]:
# Provide the paths for input JSON file, train output JSON file, and test output JSON file
input_json_path = '/media/real/data/uzbekvoice/uzbekvoice-dataset/voice_dataset.json'
train_json_path = '/media/real/data/uzbekvoice/uzbekvoice-dataset/train.json'
test_json_path = '/media/real/data/uzbekvoice/uzbekvoice-dataset/test.json'

process_and_split_json(input_json_path, train_json_path, test_json_path)

Total data size: 863237
Total valid data size: 127613
Could not find audio file: /media/real/data/uzbekvoice/clips/2a4f5725-30b6-4cf2-b6dd-1ffdf923d76c/28e79be9bbb3c0c92df1b87bffd09eff18fdab77b0c086aef02208fddb3250fe.wav
Could not find audio file: /media/real/data/uzbekvoice/clips/2a4f5725-30b6-4cf2-b6dd-1ffdf923d76c/773aa9693f6785e48b530b3ac61991851c6730eb8506c4e50d891f82c54510eb.wav
Could not find audio file: /media/real/data/uzbekvoice/clips/2a4f5725-30b6-4cf2-b6dd-1ffdf923d76c/583266af2a9051acb380b6bd18ee4a8f5df4a391766e5b7e86c431b535868a41.wav
Could not find audio file: /media/real/data/uzbekvoice/clips/2a4f5725-30b6-4cf2-b6dd-1ffdf923d76c/304881f83b3f3ecdfc6e27d3bd293e3faf972aae0337167f63718aed5e3d87a9.wav
Could not find audio file: /media/real/data/uzbekvoice/clips/2a4f5725-30b6-4cf2-b6dd-1ffdf923d76c/73d4ce11bd6a180db00196fea75cac8a8ee0140d1cde4d31a4043437d3aa74fb.wav
Could not find audio file: /media/real/data/uzbekvoice/clips/2a4f5725-30b6-4cf2-b6dd-1ffdf923d76c/3bf0322a8309b